In [2]:
import FlockModel

Viscek:
Stated briefly as rules, and in $\textbf{order of decreasing precedence}$, the behaviors that lead to simulated flocking are:
1. Collision Avoidance: avoid collisions with nearby
flockmates
2. Velocity Matching: attempt to match velocity with nearby
flockmates
3. Flock Centering: attempt to stay close to nearby flockmates 

In [ ]:
phenotypes = ["4"]
angles = [np.pi, np.pi/2, np.pi/3, np.pi/4]

for angle in angles:
    for phenotype in phenotypes:
        model = Model(dt = 0.1, maxtime = 100, noise = 0.01, phenotype = phenotype, angle = angle)
        model.run(), 
        model.animate(name = str(phenotype) + str(angle))

${\displaystyle \operatorname {distance} (P_{1},P_{2},(x_{0},y_{0}))={\frac {|(x_{2}-x_{1})(y_{1}-y_{0})-(x_{1}-x_{0})(y_{2}-y_{1})|}{\sqrt {(x_{2}-x_{1})^{2}+(y_{2}-y_{1})^{2}}}}.}$

# Create CSV of positions for blender (TEST)

In [49]:
phenotypes = ["1", "2", "3", "4", "5", "6", "7", "8", "Viscek", "Random"]
for phenotype in phenotypes:

    model = Model(dt = 0.25, maxtime = 100, noise = 0.01, phenotype = phenotype)
    model.run(), 
    model.blender_csv()

# Connected correlation function test

In [3]:
phenotypes = ["Viscek"]
for phenotype in phenotypes:
    model = FlockModel.Model(dt = 0.25, maxtime = 20, noise = 0.01, phenotype = phenotype, density=0.02)
    model.run()
    model.animate(name="correlation_test")

In [18]:
# Get velocities and positions at a certain point
vel = []
dim_vel = []
pos = []
frame = 100

for a in model.agents:
    vel.append(a.vel[frame])
    pos.append(a.pos[frame])

print("Velocities:")
for v in vel:
    print(v)

print()
print("Positions:")
for p in pos:
    print(p)

print()
print("Average velocity")
avg_vel = sum(vel)/len(vel)
print(avg_vel)

print()
print("Dimensionless velocity fluctuation")
denom = sum([np.linalg.norm(v-avg_vel)**2 for v in vel])/len(vel)
dim_vel = [(v-avg_vel)/denom for v in vel]
for v in dim_vel:
    print(v)

Velocities:
[ 0.92109952 -0.38932721]
[ 0.6407082  -0.76778448]
[ 0.92552154 -0.37869497]
[0.85677611 0.51568857]
[0.86070294 0.5091075 ]

Positions:
[8.80871417 2.16420908]
[9.5266738  3.11498382]
[8.04743847 1.88985906]
[3.22407221 4.98044335]
[2.85682943 4.63826681]

Average velocity
[ 0.84096166 -0.10220212]

Dimensionless velocity fluctuation
[ 0.28383638 -1.01695443]
[-0.70926801 -2.35739383]
[ 0.29949853 -0.97929659]
[0.05601242 2.18847702]
[0.06992068 2.16516784]


In [9]:
pred = FlockModel.Predator()
print(pred.update_pos)

<bound method Agent.update_pos of <FlockModel.Predator object at 0x7f58fc695340>>
